In [ ]:
import io, os, collections
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol(): 
  N, D = map(int, input().split())
  DQ = collections.deque([0])
  DP = [0] * N #i번째 까지 점수의 최대값
  answer = -float("inf")
  for i, v in enumerate(map(int, input().split())) :
    if DQ and DQ[0] < i - D : DQ.popleft()
    DP[i] = max(DP[DQ[0]] + v, v) #현재 징검다리부터 시작하는 경우 vs 이전의 최대값을 가지는 징검다리를 밟고 건너온
    answer = max(answer, DP[i])
    while DQ and DP[DQ[-1]] <= DP[i] : DQ.pop() #front에 최대값을 유지시키도록 만든다.
    DQ.append(i)

  print(answer)

sol()

### 풀이
- DP[i] = i번째 징검다리까지 고려했을 때 얻을 수 있는 최대 점수.
  - 덱은 `현재 값을 제외한 길이 D의 범위에서 얻을 수 있는 최대값`을 유지할 수 있도록 한다.
  - 이때 현재 값을 제외시키기 위해 L[i]를 enqueue하는 작업을 정답 후보를 계산한 다음에 수행한다.
- `i`번째 징검다리를 밟으려 할 때, 이전에 밟은 징검다리를 `j`라고 한다면
  - 이전에 밟아야 할 징검다리의 후보는 `DP[j] + L[i]` 단, $i-D \le j \le i-1$ 사이에 있을 것이며 이중 최대값을 가지는 징검다리를 밟으면 된다.
  - 이 부분은 단조 큐 테크닉으로 구하여 DQ[0]에 나오도록 만들 수 있다.
- 그러나 L은 음수가 포함되어있고, 어디서든 시작할 수 있기 때문에 이전 다리를 밟아서 누적하는 것이 해가 아닐 수 있다.
  - 현재 징검다리부터 시작하는 경우도 고려해야 한다.
  - 따라서 DP[i] = max(`이전 다리를 밟아와서 누적하는 경우`, `현재 다리부터 시작하는 경우`) = `max(DP[DQ[0], L[i])`이다.
- 그리고 어디서든지 끝낼 수도 있기 때문에, 해는 징검다리를 밟는 도중에 나올 수 있다. 
  - 따라서 `DP[i]`를 갱신할 때마다 그 최대값을 갱신시킨다.
  - L을 모두 순회하고 남은 최대값이 정답이 된다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

class ST:
  def __init__(self, L, f, default=0):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, i):
    self[i] = self._def

  def __getitem__(self, i):
    return self.L[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.L[i] = v
    i >>= 1
    while i:
      self.L[i] = self._f(self.L[2 * i], self.L[2 * i + 1])
      i >>= 1

  def __len__(self):
    return self._len

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

def sol() :
  N, D = map(int, input().split())
  L = [*map(int, input().split())]

  st = ST(L, max, -1e9)
  for i in range(N) :
    x = max(st[i], st.query(max(0, i - D), i) + st[i])
    st[i] = x
  print(st.query(0, N))
  
sol()

- max 세그먼트를 이용한 풀이